In [1]:
import pandapower as pp
import pandapower.networks
import numpy as np
import pandas as pd

In [2]:
#if network == 'IEEE-14' then
net = pp.networks.case14()
#else if network == 'IEEE-30' then
#net = pandapower.networks.case30()
#else if network == 'IEEE-57' then
#net = pandapower.networks.case57()
#else if network == 'IEEE-118' then
#net = pandapower.networks.case118()

In [3]:
# if user selects data generation strategy 1
pp.rundcpp(net)
# state vector x_base
x_base = net.res_bus.va_degree.to_numpy()
# number of discrete time steps
T = 100 # user argument
# Dx ~ N(0,Cov_Mat)
mean = np.zeros(x_base.shape)
cov = np.eye(x_base.shape[0])
delta_x_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
x_base_mat = np.repeat(x_base.reshape((x_base.shape[0],-1)), T, axis=1)
x_t_mat = x_base_mat + delta_x_mat

In [4]:
# P_from_node_i_to_node_j
# Pij = (1/bij)*(x[i]-x[j])
# H[line_id,i] = 1/bij
# H[line_id,j] = -1/bij
A_real = np.real(net._ppc['internal']['Bbus'].A)
H = np.zeros((net.line.shape[0]+net.trafo.shape[0], net.bus.shape[0]))
for i in range(0, net.line.shape[0]):
    H[i, net.line.from_bus.values[i]] = 1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]
    H[i, net.line.to_bus.values[i]] = -1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]

for i in range(net.line.shape[0]+1, net.line.shape[0]+net.trafo.shape[0]):
    H[i, net.trafo.hv_bus.values[i-net.line.shape[0]]] = 1/A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]
    H[i, net.trafo.lv_bus.values[i-net.line.shape[0]]] = -1/A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]

In [5]:
#z_t = H @ x_t + noise
mean = np.zeros(H.shape[0])
cov = np.eye(H.shape[0])/10
noise_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
z_t_mat = np.matmul(H, x_t_mat) + noise_mat
z_t_mat.shape

(20, 100)

In [163]:
# target with matching pursuit 
from sklearn.linear_model import OrthogonalMatchingPursuit

I = [i for i in range(0,H.shape[1])]
I_fixed = [0,4] # 0 always there, but "1,2,3" could have been "2,4,9"
I_free = [i for i in I if I.index(i) not in I_fixed]

H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])
c = np.zeros((H.shape[1],))
for i in I_fixed[1:]:
    c[i] = 0.1
y = B_s @ H[:,I_fixed] @ c[I_fixed]

for n in range(1,H.shape[0]+1):
    print(n)
    # Bs*a=y, solve for a where cardinality(a) = n_nonzero_coefs
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n, normalize=False)
    omp.fit(B_s, y)
    a = omp.coef_
    # a = H*c => c = (H^T * H)^(-1) * H^T * a
    resulting_c = np.linalg.inv(np.transpose(H[:,1:]) @ H[:,1:]) @ np.transpose(H[:,1:]) @ a
    resulting_c = np.hstack((0,resulting_c))
    print((c[I_fixed]-resulting_c[I_fixed]).reshape(-1,1))
    if np.abs(np.max(c[I_fixed]-resulting_c[I_fixed])) < 1e-4:
        break

1
[[0.        ]
 [0.00770795]]
2
[[0.00000000e+00]
 [2.35922393e-16]]


In [164]:
# Targeted least effort based on norm-1
# min norm-1(a) with respect to a and c, where a is in z+a and c is in x+c
# subject to:
# a = H*c -> B_s*a = y
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1

# input params: H, z, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

# new material for targerted with norm 1
I = [i for i in range(0,H.shape[1])]
I_fixed = [0,k] # 0 always there, but "1,2,3" could have been "2,4,9"
# I_fixed = [0, k]
I_free = [i for i in I if I.index(i) not in I_fixed]

# new material for targerted with norm 1
H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])

m = Model('targerted least effort with norm 1')

# define decision variables: a and c vectors

a  = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c  = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]

a_positive = [m.addVar(lb=0.0,name='a_pos%d' % i) for i in range(H.shape[0])]
a_negative = [m.addVar(ub=0.0,name='a_neg%d' % i) for i in range(H.shape[0])]

m.update()

# constraint: a[i] = a_positive[i] - a_negative[i]
# not a constraint: |a[i]| = a_positive[i] + a_negative[i]
for i in range(0,H.shape[0]):
    m.addConstr(a[i] == a_positive[i] + a_negative[i])

# 1st constraint: c[k] = delta
m.addConstr(c[0] == 0)
m.addConstr(c[k] == delta_a)

# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: B_s*a = y (replacing a = H*c)
c_aux = [c[i] for i in I_fixed]
for i in range(0,H.shape[0]):
    #m.addConstr(a[i] == np.matmul(H,c)[i])
    m.addConstr(np.matmul(B_s,a)[i] == (B_s @ H[:,I_fixed] @ c_aux)[i])
    
# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(z[0]-z[1] for z in zip(a_positive, a_negative)), GRB.MINIMIZE)

m.update()

m.optimize()

#least_effort_a_norm_1 = m.getVars()[0:H.shape[0]]
targeted_least_effort_a_norm_1 = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
targeted_least_effort_a_norm_1 = np.asarray(targeted_least_effort_a_norm_1)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 110 rows, 74 columns and 422 nonzeros
Model fingerprint: 0x779edeee
Coefficient statistics:
  Matrix range     [9e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+02]
Presolve removed 104 rows and 59 columns
Presolve time: 0.01s
Presolved: 6 rows, 15 columns, 90 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2304000e-02   3.227991e-03   0.000000e+00      0s
       1    2.8221000e-02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.822100000e-02


In [165]:
# Targeted least effort based on big-M, where M is a hyper-parameter
# min sum(y) with respect to a, c, and y, where a is in z+a and c is in x+c
# subject to:
# a = H*c -> B_s*a = y
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1
# y[i] where i going from 0 to #meas-1 and y[i] is integer
# -y[i]*M <= a[i] <= y[i]*M
# if y[i] = 0, then 0 <= a[i] <= 0
# if y[i] = 1, then -M <= a[i] <= M

# input params: H, z, M, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

M = 10**4

m = Model('targeted least effort with big-M')

# new material for targerted with norm 1
I = [i for i in range(0,H.shape[1])]
I_fixed = [0,k] # 0 always there, but "1,2,3" could have been "2,4,9"
# I_fixed = [0, k]
I_free = [i for i in I if I.index(i) not in I_fixed]

# new material for targerted with norm 1
H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])

# define decision variables: a and c vectors

a = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]
y = [m.addVar(vtype=gurobipy.GRB.BINARY, name='y%d' % i) for i in range(H.shape[0])]

m.update()

# -y[i]*M <= a[i] <= y[i]*M
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= y[i]*M)
    m.addConstr(a[i] >= -y[i]*M)

# 1st constraint: c[k] = delta
m.addConstr(c[0] == 0)
m.addConstr(c[k] == delta_a)


# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: B_s*a = y (replacing a = H*c)
c_aux = [c[i] for i in I_fixed]
for i in range(0,H.shape[0]):
    #m.addConstr(a[i] == np.matmul(H,c)[i])
    m.addConstr(np.matmul(B_s,a)[i] == (B_s @ H[:,I_fixed] @ c_aux)[i])
    
# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(y), GRB.MINIMIZE)

m.update()

m.optimize()

targeted_least_effort_a_big_M = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
targeted_least_effort_a_big_M = np.asarray(targeted_least_effort_a_big_M)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 130 rows, 54 columns and 442 nonzeros
Model fingerprint: 0xfe6dab40
Variable types: 34 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [9e-04, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+02]
Presolve removed 111 rows and 25 columns
Presolve time: 0.00s
Presolved: 19 rows, 29 columns, 57 nonzeros
Variable types: 13 continuous, 16 integer (16 binary)
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [158]:
# incomplete knowledge with modal decomposition (to be revisited)
from sklearn.decomposition import FastICA

z = z_t_mat[:,0].reshape((-1,1))
ica = FastICA(n_components=None,random_state=0,whiten='unit-variance', fun='logcosh')
y = ica.fit_transform(z) # where z_t_mat is a snapshot from a single point in time
G = ica.mixing_

#if np.max(np.abs(z - (np.dot(y, G.T) + ica.mean_))) > 1e-4:
       # raise an error and say that attack is not feasible

sigma_2 = 0.00001 # to be revisited
mean = np.zeros(y.shape[0])
cov = np.eye(y.shape[0])*sigma_2
delta_y = np.random.multivariate_normal(mean, cov).reshape((-1,1))
z_perturbed = z + np.dot(y+delta_y, G.T) + ica.mean_